In [1]:
import numpy as np
import tensorflow_addons as tfa

batch = 64
emb_dim = 1024

np.random.seed(1234)
emb1 = np.random.rand(batch,emb_dim).astype(np.float32)
np.random.seed(2345)
emb2 = np.random.rand(batch,emb_dim).astype(np.float32)
emb3 = np.concatenate([emb1, emb2], axis=0)
margin = 0.3
labels1 = np.arange(batch)
labels2 = np.concatenate((labels1,labels1), axis=0)
print(emb1.shape)
print(emb2.shape)
print(emb3.shape)
print(labels1.shape)
print(labels2.shape)

(64, 1024)
(64, 1024)
(128, 1024)
(64,)
(128,)


In [0]:
def np_distance_metric(embedding, squared=True):
    """
    Args:
        x: float32, with shape [m, d], (batch_size, d)
        y: float32, with shape [n, d], (batch_size, d)
    Returns:
        dist: float32, with shape [m, n], (batch_size, batch_size)
    """
    # |x-y|^2 = x^2 - 2xy + y^2
    xy = np.matmul(embedding, np.transpose(embedding))
    square_norm = np.diag(xy)
    xx = np.expand_dims(square_norm, 0)
    yy = np.expand_dims(square_norm, 1)
    distances = np.add(xx, yy) - 2.0 * xy
    '''
    (batch_size,1)-(batch_size,batch_size): Equivalent to each column operation
    (batch_size,batch_size)+(1,batch_size): Equivalent to each row operation
    '''
    # Deal with numerical inaccuracies. Set small negatives to zero.
    distances = np.maximum(distances, 0.0)
    # Get the mask where the zero distances are at.
    error_mask = np.less_equal(distances, 0.0).astype(np.float32)

    if not squared:
        # Because the gradient of sqrt is infinite when distances == 0.0 (ex: on the diagonal)
        # we need to add a small epsilon where distances == 0.0
        distances = np.sqrt(distances + error_mask * 1e-16)

    # Undo conditionally adding 1e-16.
    distances = np.multiply(distances, np.logical_not(error_mask),)

    num_data = np.shape(embedding)[0]
    # Explicitly set diagonals to zero.
    mask_offdiagonals = np.ones_like(distances) - np.diag(np.ones([num_data]))
    distances = np.multiply(distances, mask_offdiagonals)
    return distances

In [0]:
def np_masked_minimum(data, mask, dim=1):
    """Computes the axis wise minimum over chosen elements.
    Args:
      data: float32, with shape [n, m], (batch_size, batch_size)
      mask: boolean, with shape [n, m], (batch_size, batch_size)
      dim: int, the dimension which want to compute the minimum.
    Returns:
      masked_minimums: float32, with shape [n, 1], (batch_size, batch_size)
    """
    axis_maximums = np.max(data, dim, keepdims=True)
    masked_minimums = (np.min(np.multiply(data - axis_maximums, mask), dim, keepdims=True) + axis_maximums)
    return masked_minimums
def np_masked_maximum(data, mask, dim=1):
    """Computes the axis wise maximum over chosen elements.
    Args:
      data: float32, with shape [n, m], (batch_size, batch_size)
      mask: boolean, with shape [n, m], (batch_size, batch_size)
      dim: int, the dimension over which to compute the maximum.
    Returns:
      masked_minimums: float32, with shape [n, 1], (batch_size, batch_size)
    """
    axis_minimums = np.min(data, dim, keepdims=True)
    masked_maximums = (np.max(np.multiply(data - axis_minimums, mask), dim, keepdims=True) + axis_minimums)
    return masked_maximums

In [0]:
'''
batch hard triplet loss of a batch
------------------------------------
Args:
    labels:     Label Data, shape = (batch_size,1)
    embedding:  embedding vector, shape = (batch_size, vector_size)
    margin:     margin, scalar
    soft::     	use log1p or not, boolean
Returns:
    triplet_loss: scalar, for one batch
'''
def np_triplet_batch_hard(labels, embedding, margin, soft):
    lshape = np.shape(labels)
    labels = np.reshape(labels, [lshape[0], 1])
    # Build pairwise squared distance matrix.
    pdist_matrix = np_distance_metric(embedding, squared=True)

    # Build pairwise binary adjacency matrix.
    adjacency = np.equal(labels, np.transpose(labels)).astype(np.float32)
    # Invert so we can select negatives only.
    adjacency_not = np.logical_not(adjacency).astype(np.float32)

    # hard negatives: smallest D_an.
    hard_negatives = np_masked_minimum(pdist_matrix, adjacency_not)

    batch_size = np.size(labels)
    mask_positives = adjacency - np.diag(np.ones([batch_size]))
    # hard positives: largest D_ap.
    hard_positives = np_masked_maximum(pdist_matrix, mask_positives)
    if soft:
        triplet_loss = np.log1p(np.exp(hard_positives - hard_negatives))
    else:
        triplet_loss = np.maximum(hard_positives - hard_negatives + margin, 0.0)

    # Get final mean triplet loss
    triplet_loss = np.mean(triplet_loss)

    return triplet_loss

In [5]:
soft = True
tfa_triplet = tfa.losses.TripletHardLoss(0.3, soft)
print(tfa_triplet(labels2, emb3))
print(np_triplet_batch_hard(labels2,emb3,0.3, soft))

tf.Tensor(13.9740715, shape=(), dtype=float32)
13.974051951069422


In [6]:
embedding = np.array([[-2., 0., 3.],[-1., 3., 2.],[-3., 1., 6.],[2., -1., -2.]]).astype(np.float32)
labels = np.array([1,0,1,0]).astype(np.float32)
print(embedding.shape)
print(labels.shape)
# a = tf.cast(a, dtype=tf.dtypes.float32)
squared = True
margin = 0.3

(4, 3)
(4,)


In [0]:
'''
semi-hard batch triplet loss of a batch
------------------------------------
Args:
    labels:     label data, shape = (batch_size,1)
    embedding:  embedding vector, shape = (batch_size, vector_size)
    margin:     margin, scalar
Returns:
    triplet_loss: scalar, for one batch
'''
def np_triplet_batch_semihard(labels, embedding, margin):
    # Reshape label tensor to [batch_size, 1].
    lshape = np.shape(labels)
    labels = np.reshape(labels, [lshape[0], 1])
    # Build pairwise squared distance matrix.
    pdist_matrix = np_distance_metric(embedding, squared=True)

    # Build pairwise binary adjacency matrix.
    adjacency = np.equal(labels, np.transpose(labels))
    # Invert so we can select negatives only.
    adjacency_not = np.logical_not(adjacency)

    batch_size = np.size(labels)
    # Compute the mask.
    pdist_matrix_tile = np.tile(pdist_matrix, [batch_size, 1])
    mask = np.logical_and(np.tile(adjacency_not, [batch_size, 1]),
                          np.greater(pdist_matrix_tile, np.reshape(np.transpose(pdist_matrix), [-1, 1])),)
    mask_final = np.reshape(np.greater(np.sum(mask.astype(np.float32), 1, keepdims=True),
                                       0.0,),
                            [batch_size, batch_size],)
    mask_final = np.transpose(mask_final)

    adjacency_not = adjacency_not.astype(np.float32)
    mask = mask.astype(np.float32)

    # negatives_outside: smallest D_an where D_an > D_ap.
    negatives_outside = np.reshape(np_masked_minimum(pdist_matrix_tile, mask), [batch_size, batch_size])
    negatives_outside = np.transpose(negatives_outside)

    # negatives_inside: largest D_an.
    negatives_inside = np.tile(np_masked_maximum(pdist_matrix, adjacency_not), [1, batch_size])

    semi_hard_negatives = np.where(mask_final, negatives_outside, negatives_inside)

    loss_mat = np.add(margin, pdist_matrix - semi_hard_negatives)

    mask_positives = adjacency.astype(np.float32) - np.diag(np.ones([batch_size]))

    # In lifted-struct, the authors multiply 0.5 for upper triangular
    #   in semihard, they take all positive pairs except the diagonal.
    num_positives = np.sum(mask_positives)

    triplet_loss = np.true_divide(np.sum(np.maximum(np.multiply(loss_mat, mask_positives), 0.0)),num_positives,)

    return triplet_loss

In [8]:
# print(embedding.shape)
# print(labels.shape)
tfa_triplet = tfa.losses.TripletSemiHardLoss(0.3)
print(tfa_triplet(labels2, emb3))
print(np_triplet_batch_semihard(labels2,emb3,0.3))

tf.Tensor(0.15625882, shape=(), dtype=float32)
0.1562326908111572
